In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import numpy as np
import pickle

test_data = np.load("/content/drive/My Drive/DataShuffled/training_images", allow_pickle = True)
test_labels = np.load("/content/drive/My Drive/DataShuffled/training_subjects", allow_pickle = True)
train_data = np.load("/content/drive/My Drive/DataShuffled/test_images", allow_pickle = True)
train_labels = np.load("/content/drive/My Drive/DataShuffled/test_subjects", allow_pickle = True)

In [ ]:
# Reduce size of image from 360x102 to 360x100 by extracting two uttermost bottom pixels.

test_data_r = np.zeros((1120, 100, 360))
train_data_r = np.zeros((1120, 100, 360))

for i in range(test_data.shape[0]):
  test_data_r[i, :,:] = test_data[i, 0:100, :]

for i in range(train_data.shape[0]):
  train_data_r[i, :,:] = train_data[i, 0:100, :]

In [ ]:
import cv2
import numpy as np
import pylab as pl
import glob
import pandas as pd
import math

# define gabor filter bank with different orientations and at different scales
def build_filters():
  num = 0
  theta = 0
  lamda = 2
  kernels = []
  data = np.zeros((24, 4))
  for lamda in [1.4, 1.7, 2.0, 2.3]:
    for sigma in [1.4, 1.8]:
      for gamma in [0, 1]: # Define number of thetas
        for i in [0, 2, 4, 6]:
          theta = math.pi*i/8
          gabor_label = 'Gabor' + str(num)
          ksize = 25
          kernel = cv2.getGaborKernel((ksize,ksize), sigma, theta, lamda, gamma, 0, ktype = cv2.CV_32F)
          kernels.append(kernel)
          data[num, 0] = theta
          data[num, 1] = sigma
          data[num, 2] = lamda
          data[num, 3] = gamma
          print(gabor_label, ' : theta = ', theta, ' : sigma = ', sigma, ' : lambda = ', lamda, ' : gamma = ', gamma)
          num = num + 1
  return kernels, data

#function to convolve the image with the filters
def process(img, filters):
  accum = np.zeros_like(img)
  fimg = cv2.filter2D(img, cv2.CV_8UC3, filters)
  np.maximum(accum, fimg, accum)
  return accum

In [ ]:
import matplotlib.pyplot as plt

def FeatureExtraction(image, filterBank):

  #initializing the feature vector
  f_range = len(filterBank)
  vector = []
  for m in range(f_range):
    img_check = process(image, filterBank[m])
    i = 0
    k = 0
    while i < img_check.shape[0]:
      j = 0
      while j < img_check.shape[1]:
        mean_img = img_check[i:i+25, j:j+60].mean()
        AAD = abs(img_check[i:i+25, j:j+60] - mean_img).mean()
        

        vector.append(mean_img)
        vector.append(AAD)

        j = j + 60
      i = i + 25
  return vector

In [ ]:
filters = build_filters()

In [ ]:
from tqdm import tqdm
feature_vector_train = np.zeros((1120, 2304))

for i in range(train_data.shape[0]):
  feature_vector_train[i, :] = FeatureExtraction(train_data_r[i], filters[0])

feature_vector_test = np.zeros((1118, 2304)) 

for i in (range(test_data.shape[0])):
 feature_vector_test[i, :] = FeatureExtraction(test_data_r[i], filters[0])

In [ ]:
# Shuffle train_data with labels : 

from random import shuffle

ind_list = [i for i in range(1120)]
shuffle(ind_list)
train_features  = feature_vector_train[ind_list]
y_train= train_labels[ind_list]


#Apply PCA :
from sklearn.decomposition import PCA 

pca = PCA(.99, whiten= True)
pca.fit(train_features)
train_features_pca = pca.transform(train_features)
test_features_pca = pca.transform(feature_vector_test)

## KNN

In [ ]:
#Apply Classifier
import sklearn
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score,roc_curve, auc
from sklearn.metrics import classification_report


knn = sklearn.neighbors.KNeighborsClassifier(n_neighbors=1)
knn.fit(train_features_pca, y_train)

scores = knn.predict(train_features_pca)

# Show train accuracy
print('\nPrediction accuracy:')
print('{:.2%}\n'.format(accuracy_score(y_train, scores)))

In [ ]:
# function to transform labels to one hot :

def one_hot(labels):
  one_hot = []
  for i in range(len(labels)):
    y = np.zeros((224))
    y[labels[i]]=1
    one_hot.append(y)
  one_hot = np.array(one_hot)
  return one_hot

In [ ]:
y_pred = knn.predict(test_features_pca)

# Show prediction accuracy
print('\nPrediction accuracy:')
print('{:.2%}\n'.format(accuracy_score(test_labels, y_pred)))
print('\nPrecision:')
print('{:.2%}\n'.format(precision_score(test_labels, y_pred,average='weighted')))
print('\nrecall:')
print('{:.2%}\n'.format(recall_score(test_labels, y_pred,average='weighted')))
print('\nf1 score:')
print('{:.2%}\n'.format(f1_score(test_labels, y_pred,average='weighted')))
print('\n auc:')
print('{:.2%}\n'.format(roc_auc_score(one_hot(test_labels), one_hot(y_pred), average='weighted', multi_class='ovo')))

## SVM

In [ ]:
#SVM
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold, StratifiedShuffleSplit
from sklearn.pipeline import Pipeline

pipe = Pipeline([('scaler', StandardScaler()),('classifier',SVC())])

params = dict()
params['classifier__C'] = np.logspace(-5, 5, 10)
params['classifier__kernel'] = ['linear', 'poly', 'rbf']
params['classifier__gamma'] = [0.01, 0.1, 1]

cv = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=0)

grid_search_svm = GridSearchCV(pipe, params, cv = cv)
grid_search_svm.fit(train_features_pca, y_train)

print("The best parameters are %s with an accuracy of %0.4f"%(grid_search_svm.best_params_, grid_search_svm.best_score_))


In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd
import seaborn as sn
y_pred= grid_search_svm.predict(test_features_pca)
cm = confusion_matrix(y_pred, test_labels)

# Svm metrics 

print('\nPrediction accuracy:')
print('{:.2%}\n'.format(accuracy_score(test_labels, y_pred)))
print('\nPrecision:')
print('{:.2%}\n'.format(precision_score(test_labels, y_pred,average='weighted')))
print('\nrecall:')
print('{:.2%}\n'.format(recall_score(test_labels, y_pred,average='weighted')))
print('\nf1 score:')
print('{:.2%}\n'.format(f1_score(test_labels, y_pred,average='weighted')))
print('\n auc:')
print('{:.2%}\n'.format(roc_auc_score(one_hot(test_labels), one_hot(y_pred), average='weighted', multi_class='ovo')))